In [1]:
from itertools import chain
from random import random

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

from society.action import Action
from society.agent import Agent, TrainableAgent
from society.simulations.weighted import WeightedNetworkSimulation
from society.strategies.gameplay.constant import AllC, AllD
from society.strategies.gameplay.qlearning import TabularQLearningGameplayStrategy
from society.strategies.gameplay.random import RandomGameplayStrategy
from society.strategies.gameplay.tft import TitForTat
from society.visualisation.network import *

plt.rcParams["figure.figsize"] = (10, 6)

In [2]:
def generate_population(communities, size):
    population = communities * size

    agents = [
        Agent(TabularQLearningGameplayStrategy(lookback=3), i, population)
        for i in range(population)
    ]

    G = nx.connected_caveman_graph(communities, size)

    weights_matrix = np.zeros((population, population))
    for u, v, d in G.edges(data=True):
        weights_matrix[u, v] = weights_matrix[v, u] = 1.0
        try:
            d["weight"] = weights_matrix[u, v]
        except:
            d["weight"] = 0

    return agents, weights_matrix, G

In [3]:
def compute_policies(agents, rankings):
    policies = [
        (agents[partner[0]].gameplay_strategy._q_table.argmax(axis=-1), partner[1])
        for partner in rankings
    ]

    return [
        (
            "".join(
                [
                    ("C", "D")[policy[0][i, j, k]]
                    for i in range(4)
                    for j in range(4)
                    for k in range(4)
                ]
            ),
            policy[1],
        )
        for policy in policies
    ]

def calculate_cooperativeness(history):
    count = history.count(Action.COOPERATE)

    return count / len(history)

In [4]:
ACTION_LABELS = ["(C, C)", "(C, D)", "(D, C)", "(D, D)"]
POPULATION = (2, 32)
ROUNDS = 20_000

cumulative_reward_results = []
cooperativeness_results = []

for run in range(20):
    # Generate a new population
    agents, weights_matrix, G = generate_population(*POPULATION)

    # Run a number of rounds
    sim = WeightedNetworkSimulation(agents, weights_matrix)
    sim.reset()
    for i in tqdm(range(ROUNDS), desc=f"Run {run + 1}"):
        sim.play_round()

    # Rank agents by cumulative reward
    cumulative_rewards_matrix = [
        [sum(r) if len(r) > 0 else 0 for r in agent] for agent in sim.rewards
    ]
    cumulative_reward_rankings = sorted(
        [(i, sum(cumulative_rewards_matrix[i])) for i in range(len(agents))],
        key=lambda x: x[1],
        reverse=True,
    )

    tqdm.write(
        f"BEST REWARD: {cumulative_reward_rankings[0][1]:<28} WORST REWARD: {cumulative_reward_rankings[-1][1]}"
    )

    cumulative_reward_results.append(compute_policies(agents, cumulative_reward_rankings))

    # Rank agents by cooperativeness
    cooperativeness_rankings = sorted(
        [
            (i, calculate_cooperativeness(list(chain(*history))))
            for i, history in enumerate(sim.action_histories)
        ],
        key=lambda x: x[1],
        reverse=True,
    )

    tqdm.write(
        f"BEST COOPERATOR: {cooperativeness_rankings[0][1]:<24} WORST COOPERATOR: {cooperativeness_rankings[-1][1]}"
    )

    cooperativeness_results.append(compute_policies(agents, cooperativeness_rankings))


Run 1:   0%|          | 0/20000 [00:00<?, ?it/s]

BEST REWARD: 139310                       WORST REWARD: 111860
BEST COOPERATOR: 0.9479709418837675       WORST COOPERATOR: 0.5437295583352059


Run 2:   0%|          | 0/20000 [00:00<?, ?it/s]

BEST REWARD: 140574                       WORST REWARD: 110802
BEST COOPERATOR: 0.949617220123285        WORST COOPERATOR: 0.5574617506897417


Run 3:   0%|          | 0/20000 [00:00<?, ?it/s]

...

BEST REWARD: 138141                       WORST REWARD: 112138
BEST COOPERATOR: 0.9493331998899092       WORST COOPERATOR: 0.6007475791480608


Run 19:   0%|          | 0/20000 [00:00<?, ?it/s]

BEST REWARD: 138004                       WORST REWARD: 109183
BEST COOPERATOR: 0.948057485390384        WORST COOPERATOR: 0.6069377153988312


Run 20:   0%|          | 0/20000 [00:00<?, ?it/s]

BEST REWARD: 138585                       WORST REWARD: 111721
BEST COOPERATOR: 0.949023833989123        WORST COOPERATOR: 0.6037480921760452


In [5]:
cumulative_reward_policy_ranks = {}

for result in cumulative_reward_results:
    for rank, (policy, score) in enumerate(result):
        if policy not in cumulative_reward_policy_ranks:
            cumulative_reward_policy_ranks[policy] = []

        cumulative_reward_policy_ranks[policy].append(rank + 1)

mean_cumulative_reward_policy_ranks = {policy: np.mean(ranks) for policy, ranks in cumulative_reward_policy_ranks.items()}

for policy in sorted(mean_cumulative_reward_policy_ranks, key=lambda x: mean_cumulative_reward_policy_ranks[x]):
    print(f"{mean_cumulative_reward_policy_ranks[policy]:<24} {policy}")

1.0                      DCCCDCCCDCCCDCCCDCCCCDCCCCCCCCCCDCCCDCCCCCCCDCCCCCCCDCCDDCCCCCCC
1.0                      DDCDDCDCDCCCCCCCCCCCCCCCDCCCDCCCDCCCDCDCCCCCDCCCDCCCCCCCCCCCDCCC
1.0                      DCCCDCCCDCCCDCCCCCCCCCDCCCCCCDDCCCCCCCDCDCCCCCCCDCCCCCCCDCCCCCCC
1.0                      DCDCCCCCCCDDCCCCCCCCCDCCCCCCCCCCCCDCCCCCCCDCCCCCCCDCCCCCCCCCCCCC
1.0                      DCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCDCDCCCCCCDCCCCCCCCCCDCCCC
1.0                      DCCCCCCCDCDCDCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCDCCCDCCCDCCCCCCCCCCC
1.0                      DCCCDDCCDCDCCDCCCCCCCCCCCCCCCCCCDCCCDCDCDCCDCCCDCCCCCCCCCCCCCCDC
1.0                      DCCCDCCCDCCCCCCCCCCCCCCCCCCCCCDCDCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCC
1.0                      DDDCCCCCCCDCCCDCCCCCCCCCCCCCCCCCDCCCCDCCCCDCCCCCDCDCCCCDCCCCCCCC
1.0                      DCCCCCCCDCCCCCCDCCCCCCCCCCCCDCCCDCCCDCCCCCCCCCCCCCCCCCCCCCCCDCCC
1.0                      DCCCCCCCDCCCCCCCDCCCCCCCCCCCCCCCCCCCCCDCDCCCCCCCCCCCCCCCCCCCCCCC
1.0       

In [6]:
cooperativeness_policy_ranks = {}
cooperativeness_policy_scores = {}

for result in cooperativeness_results:
    for rank, (policy, score) in enumerate(result):
        if policy not in cooperativeness_policy_ranks:
            cooperativeness_policy_ranks[policy] = []
            cooperativeness_policy_scores[policy] = []

        cooperativeness_policy_ranks[policy].append(rank + 1)
        cooperativeness_policy_scores[policy].append(score)        

mean_cooperativeness_policy_ranks = {policy: (np.mean(ranks), len(ranks)) for policy, ranks in cooperativeness_policy_ranks.items()}

mean_cooperativeness_policy_scores = {policy: (np.mean(scores), len(scores)) for policy, scores in cooperativeness_policy_scores.items()}

for policy in sorted(mean_cooperativeness_policy_ranks, key=lambda x: mean_cooperativeness_policy_ranks[x][0]):
    print(f"{mean_cooperativeness_policy_ranks[policy][0]:<24} {policy:<24} {mean_cooperativeness_policy_ranks[policy][1]:<8} {mean_cooperativeness_policy_scores[policy][0]}")

1.0                      CCCCCCCCCCCCCCCCCCCCCCCCCDDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC 1        0.9479709418837675
1.0                      CCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCC 1        0.949617220123285
1.0                      CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCD 1        0.9486491232471214
1.0                      CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCDCCCCCCCDCCCC 1        0.948576859586814
1.0                      CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCDCCCDCCC 1        0.9490515990190681
1.0                      CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCDDCCCCC 1        0.9450018766420618
1.0                      CCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCD 1        0.9497641686007336
1.0                      CCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCDCC 1        0.948629793214214
1.0                      CCCCCCCCCCCCCCCDCCCCCCCDCCCCCCCCCC

In [7]:
for policy in sorted(mean_cooperativeness_policy_scores, key=lambda x: mean_cooperativeness_policy_scores[x][0], reverse=True):
    print(f"{mean_cooperativeness_policy_ranks[policy][0]:<24} {policy:<24} {mean_cooperativeness_policy_ranks[policy][1]:<8} {mean_cooperativeness_policy_scores[policy][0]}")

1.0                      CCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCD 1        0.9497641686007336
1.0                      CCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCC 1        0.949617220123285
1.0                      DCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCDCCCDCCCCCCCCCCCCC 1        0.9495207988358674
1.0                      CCCCCCCCCCCCCCDCCCCCCCCCCCCDCCCCCCCCCCCCCCCCDDCCCCCCCCCCCCCCCCCC 1        0.9494760011005228
2.0                      DCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCCCC 1        0.9494154676258992
1.0                      CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCCCDCCCC 1        0.9492007992007993
1.0                      CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCDCCCCCCCCCDCCCDCCC 1        0.9490515990190681
1.0                      CCCCCCCCCCCCCDDCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC 1        0.949023833989123
2.0                      CCCCCCCCCCCCCDCCCCCCCCCCCCCCCCCCC